In [10]:
%load_ext rpy2.ipython

import pandas as pd
from metasub_utils.packet_parse import MetaSUBTableFactory
from capalyzer.packet_parser.experimental import umap
import warnings
from os.path import isfile

warnings.filterwarnings('ignore')

core_tabler = MetaSUBTableFactory.all_factory()

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [18]:
tbl = core_tabler.metadata.copy()

tbl = tbl.loc[tbl['control_type_coarse'].isna()]
tbl = tbl.loc[~tbl['city'].isna()]
tbl = tbl.query('city != "other"')

tbl = pd.crosstab(tbl['city'], tbl['project'])

tbl['continent'] = core_tabler.metadata.groupby('city').apply(lambda x: x['continent'].iloc[0])
tbl['continent'] = tbl['continent'].str.replace('_', ' ')
tbl['continent'] = tbl['continent'].str.title()
tbl.index = tbl.index.str.replace('_', ' ')
tbl.index = tbl.index.str.title()
tbl = tbl.set_index('continent', append=True)
tbl = tbl.reorder_levels(['continent', 'city'])
tbl = tbl.sort_index()

tbl['Other'] = tbl['PATHOMAP_WINTER'] + tbl['TIGRESS'] + tbl['OLYMPIOME']
tbl = tbl.drop(columns=['PATHOMAP_WINTER', 'TIGRESS', 'OLYMPIOME'])
tbl['Total'] = tbl['PILOT'] + tbl['CSD16'] + tbl['CSD17'] + tbl['Other']
tbl = tbl[['PILOT', 'CSD16', 'CSD17', 'Other', 'Total']]

def totals(df):
    row = df.sum()
    df.loc['Total'] = row
    return df

continent_totals = tbl.groupby(level=0).sum()
continent_totals['city'] = 'AAA Continent Total' #continent_totals.index
continent_totals = continent_totals.set_index('city', append=True)
tbl = pd.concat([tbl, continent_totals]).sort_index()
tbl

project                                 PILOT  CSD16  CSD17  Other  Total
continent          city                                                  
East Asia          AAA Continent Total     22     25   1244      0   1291
                   Hanoi                    0      0     16      0     16
                   Hong Kong                0      0    713      0    713
                   Kuala Lumpur             0      0     14      0     14
                   Sendai                   0      0     16      0     16
                   Seoul                   12      0     78      0     90
                   Shanghai                10      0      0      0     10
                   Singapore                0      0    192      0    192
                   Taipei                   0      0     94      0     94
                   Tokyo                    0     25    108      0    133
                   Tsuruoka                 0      0      4      0      4
                   Yamaguchi                0      0      9      0      9
Europe             AAA Continent Total    131    235    845     71   1282
                   Barcelona                7     96      0      0    103
                   Belfast                  0      0      0      5      5
                   Berlin                   0     55      0      0     55
                   Birmingham               0      0      0      7      7
                   Bradford                 0      0      0      4      4
                   Bury                     0      0      0      6      6
                   Eastbourne               0      0      0      6      6
                   Eden                     0      0      0      5      5
                   Edinborough              0      0      0      6      6
                   Islington                0      0      0      5      5
                   Jaywick                  0      0      0      6      6
                   Kensington               0      0      0      5      5
                   Kyiv                     0      0     96      0     96
                   Lands End                0      0      0      5      5
                   Lisbon                   0     84      0      0     84
                   London                   0      0    535      0    535
                   Marseille                0      0     16      0     16
...                                       ...    ...    ...    ...    ...
                   Sofia                    0      0     16      0     16
                   Stockholm                0      0     64      0     64
                   Swansea                  0      0      0      6      6
                   Vienna                   0      0     16      0     16
                   Zurich                   0      0     54      0     54
Middle East        AAA Continent Total      0     66     14      0     80
                   Doha                     0     66     14      0     80
North America      AAA Continent Total     28    221    158    184    591
                   Baltimore                0      0     14      0     14
                   Denver                   0     23     16      0     39
                   Fairbanks                0    100      0      0    100
                   Mexico City             10      0      0      0     10
                   Minneapolis              0      0     16      0     16
                   New York City            0     82     96    184    362
                   Sacramento              18     16      0      0     34
                   San Francisco            0      0     16      0     16
Oceania            AAA Continent Total      0     31     32      0     63
                   Auckland                 0     15      0      0     15
                   Brisbane                 0      0     16      0     16
                   Hamilton                 0     16      0      0     16
                   Honolulu                 0      0     16      0     16
Sout

In [21]:

ctrl = core_tabler.metadata.copy()

ctrl = ctrl.loc[~ctrl['control_type_coarse'].isna()]
ctrl = pd.crosstab(ctrl['control_type_coarse'], ctrl['project'])
ctrl.index.names = ['city']
ctrl['continent'] = 'Control'
ctrl = ctrl.set_index('continent', append=True)
ctrl = ctrl.reorder_levels(['continent', 'city'])
ctrl['OLYMPIOME'] = 0

ctrl['Other'] = ctrl['PATHOMAP_WINTER'] + ctrl['TIGRESS'] + ctrl['OLYMPIOME']
ctrl = ctrl.drop(columns=['PATHOMAP_WINTER', 'TIGRESS', 'OLYMPIOME'])
ctrl['Total'] = ctrl['PILOT'] + ctrl['CSD16'] + ctrl['CSD17'] + ctrl['Other']
ctrl = ctrl[['PILOT', 'CSD16', 'CSD17', 'Other', 'Total']]
ctrl

project                         PILOT  CSD16  CSD17  Other  Total
continent city                                                   
Control   background_control        0     18     49      0     67
          lab_negative_control      9      0     24      7     40
          positive_control          9      1     33      7     50

In [22]:
tbl = pd.concat([ctrl, tbl])
tbl.index = tbl.index.set_levels(tbl.index.levels[1].str.replace('AAA', ''), level=1)
tbl = tbl[['PILOT', 'CSD16', 'CSD17', 'Other', 'Total']]
tbl

project                                  PILOT  CSD16  CSD17  Other  Total
continent          city                                                   
Control            background_control        0     18     49      0     67
                   lab_negative_control      9      0     24      7     40
                   positive_control          9      1     33      7     50
East Asia           Continent Total         22     25   1244      0   1291
                   Hanoi                     0      0     16      0     16
                   Hong Kong                 0      0    713      0    713
                   Kuala Lumpur              0      0     14      0     14
                   Sendai                    0      0     16      0     16
                   Seoul                    12      0     78      0     90
                   Shanghai                 10      0      0      0     10
                   Singapore                 0      0    192      0    192
                   Taipei                    0      0     94      0     94
                   Tokyo                     0     25    108      0    133
                   Tsuruoka                  0      0      4      0      4
                   Yamaguchi                 0      0      9      0      9
Europe              Continent Total        131    235    845     71   1282
                   Barcelona                 7     96      0      0    103
                   Belfast                   0      0      0      5      5
                   Berlin                    0     55      0      0     55
                   Birmingham                0      0      0      7      7
                   Bradford                  0      0      0      4      4
                   Bury                      0      0      0      6      6
                   Eastbourne                0      0      0      6      6
                   Eden                      0      0      0      5      5
                   Edinborough               0      0      0      6      6
                   Islington                 0      0      0      5      5
                   Jaywick                   0      0      0      6      6
                   Kensington                0      0      0      5      5
                   Kyiv                      0      0     96      0     96
                   Lands End                 0      0      0      5      5
...                                        ...    ...    ...    ...    ...
                   Sofia                     0      0     16      0     16
                   Stockholm                 0      0     64      0     64
                   Swansea                   0      0      0      6      6
                   Vienna                    0      0     16      0     16
                   Zurich                    0      0     54      0     54
Middle East         Continent Total          0     66     14      0     80
                   Doha                      0     66     14      0     80
North America       Continent Total         28    221    158    184    591
                   Baltimore                 0      0     14      0     14
                   Denver                    0     23     16      0     39
                   Fairbanks                 0    100      0      0    100
                   Mexico City              10      0      0      0     10
                   Minneapolis               0      0     16      0     16
                   New York City             0     82     96    184    362
                   Sacramento               18     16      0      0     34
                   San Francisco             0      0     16      0     16
Oceania             Continent Total          0     31     32      0     63
                   Auckland                  0     15      0      0     15
                   Brisbane                  0      0     16      0     16
                   Hamilton                  0     16      0      0     16
                   Honolu

In [23]:
print(tbl.to_latex())

\begin{tabular}{llrrrrr}
\toprule
                   & project &  PILOT &  CSD16 &  CSD17 &  Other &  Total \\
continent & city &        &        &        &        &        \\
\midrule
Control & background\_control &      0 &     18 &     49 &      0 &     67 \\
                   & lab\_negative\_control &      9 &      0 &     24 &      7 &     40 \\
                   & positive\_control &      9 &      1 &     33 &      7 &     50 \\
East Asia &  Continent Total &     22 &     25 &   1244 &      0 &   1291 \\
                   & Hanoi &      0 &      0 &     16 &      0 &     16 \\
                   & Hong Kong &      0 &      0 &    713 &      0 &    713 \\
                   & Kuala Lumpur &      0 &      0 &     14 &      0 &     14 \\
                   & Sendai &      0 &      0 &     16 &      0 &     16 \\
                   & Seoul &     12 &      0 &     78 &      0 &     90 \\
                   & Shanghai &     10 &      0 &      0 &      0 &     10 \\
                

In [24]:
tbl.to_csv('/home/dcdanko/Dropbox/Projects/Environmental Microbiomics/metasub_table1.csv')